In [1]:
# 04_Model_Building_and_Evaluation.ipynb

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (accuracy_score, f1_score, roc_auc_score,mean_squared_error, mean_absolute_error, r2_score,confusion_matrix, classification_report)
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import shap
import joblib
import warnings
warnings.filterwarnings('ignore')

# Load data and repeat feature engineering (same as in notebook 3)
stats = pd.read_csv('cleaned_stats.csv')
results = pd.read_csv('cleaned_results.csv')

# Recreate standings and df (same steps as before)
def compute_season_points(df):
    home = df.groupby(['season', 'home_team']).agg(
        home_wins=('result', lambda x: (x=='H').sum()),
        home_draws=('result', lambda x: (x=='D').sum()),
        home_goals_for=('home_goals', 'sum'),
        home_goals_against=('away_goals', 'sum')
    ).reset_index().rename(columns={'home_team': 'team'})
    away = df.groupby(['season', 'away_team']).agg(
        away_wins=('result', lambda x: (x=='A').sum()),
        away_draws=('result', lambda x: (x=='D').sum()),
        away_goals_for=('away_goals', 'sum'),
        away_goals_against=('home_goals', 'sum')
    ).reset_index().rename(columns={'away_team': 'team'})
    standings = home.merge(away, on=['season', 'team'], how='outer').fillna(0)
    standings['wins'] = standings['home_wins'] + standings['away_wins']
    standings['draws'] = standings['home_draws'] + standings['away_draws']
    standings['losses'] = 38 - standings['wins'] - standings['draws']
    standings['points'] = 3*standings['wins'] + standings['draws']
    standings['goals_for'] = standings['home_goals_for'] + standings['away_goals_for']
    standings['goals_against'] = standings['home_goals_against'] + standings['away_goals_against']
    standings['goal_diff'] = standings['goals_for'] - standings['goals_against']
    standings = standings.sort_values(['season', 'points', 'goal_diff', 'goals_for'],
                                       ascending=[True, False, False, False])
    standings['rank'] = standings.groupby('season').cumcount() + 1
    return standings

standings = compute_season_points(results)
stats['season'] = stats['season'].astype(str)
standings['season'] = standings['season'].astype(str)

df = stats.merge(standings[['season', 'team', 'rank', 'points', 'goals_for', 'goals_against',
                            'wins', 'draws', 'losses']],
                 on=['season', 'team'], how='left', suffixes=('', '_target'))
df['top4'] = (df['rank'] <= 4).astype(int)
df['relegated'] = (df['rank'] >= 18).astype(int)

# Sort and create lag features
df = df.sort_values(['team', 'season_start'])
feature_cols = [c for c in stats.columns if c not in
                ['team', 'season', 'season_start', 'wins', 'losses', 'draws',
                 'points', 'goal_diff', 'total_matches', 'strength', 'cluster']]
lagged = df.groupby('team')[feature_cols].shift(1).add_suffix('_lag1')
df = pd.concat([df, lagged], axis=1)
for col in feature_cols:
    df[col + '_ma2'] = df.groupby('team')[col].transform(lambda x: x.rolling(2, min_periods=1).mean().shift(1))
df = df.dropna(subset=df.filter(like='_lag1').columns, how='all').reset_index(drop=True)

# Split: train on <=2014, valid 2015, test >=2016
train = df[df['season_start'] <= 2014]
valid = df[df['season_start'] == 2015]
test = df[df['season_start'] >= 2016]

# Combine train and valid for final model training (after hyperparameter tuning)
train_val = pd.concat([train, valid])

# Function to get features and target
def get_X_y(df, target_col, classification=True):
    feat_pattern = ['_lag1$', '_ma2$']
    feat_cols = [c for c in df.columns if any(p in c for p in feat_pattern)]
    X = df[feat_cols].copy()
    le = LabelEncoder()
    X['team_encoded'] = le.fit_transform(df['team'])
    y = df[target_col].copy()
    if classification and y.dtype not in ['int64', 'int32']:
        y = y.astype(int)
    return X, y, feat_cols, le


C:\Users\phill\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 4.1 Hyperparameter Optimization (Example for one target) 

# We'll demonstrate tuning for 'goals_scored' regression using XGBoost
target_info = {'name': 'goals_scored', 'type': 'regression', 'col': 'goals'}

X_train, y_train, _, le_train = get_X_y(train, target_info['col'])
X_val, y_val, _, _ = get_X_y(valid, target_info['col'])
X_train_val, y_train_val, _, _ = get_X_y(train_val, target_info['col'])

# Scale for linear models if needed, but we'll tune tree model
import optuna

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'random_state': 42
    }
    model = xgb.XGBRegressor(**params, verbosity=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30, show_progress_bar=True)

print("Best params:", study.best_params)
print("Best RMSE on validation:", study.best_value)

# Retrain on train+valid with best params
best_params = study.best_params
best_model = xgb.XGBRegressor(**best_params, random_state=42, verbosity=0)
best_model.fit(X_train_val, y_train_val) 

[I 2026-02-23 18:07:28,179] A new study created in memory with name: no-name-be2cae7f-113f-4f0b-b615-4194c42a0f1f
Best trial: 1. Best value: 19.0511:   7%|███▏                                           | 2/30 [00:00<00:01, 14.21it/s]

[I 2026-02-23 18:07:28,247] Trial 0 finished with value: 19.178068446174965 and parameters: {'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.22174247549549753, 'subsample': 0.865504551748117, 'colsample_bytree': 0.6086079925236536, 'reg_alpha': 0.009121570553547675, 'reg_lambda': 0.2770022784727807}. Best is trial 0 with value: 19.178068446174965.
[I 2026-02-23 18:07:28,318] Trial 1 finished with value: 19.051068389649682 and parameters: {'n_estimators': 226, 'max_depth': 10, 'learning_rate': 0.07196583084597512, 'subsample': 0.9257918448765557, 'colsample_bytree': 0.7303099017162146, 'reg_alpha': 0.4286996110329887, 'reg_lambda': 0.896427624447369}. Best is trial 1 with value: 19.051068389649682.


Best trial: 3. Best value: 18.9135:  10%|████▋                                          | 3/30 [00:00<00:01, 14.21it/s]

[I 2026-02-23 18:07:28,369] Trial 2 finished with value: 19.111472537938027 and parameters: {'n_estimators': 136, 'max_depth': 6, 'learning_rate': 0.19426334124155858, 'subsample': 0.6943306397104628, 'colsample_bytree': 0.6605775078726291, 'reg_alpha': 0.13321429836887433, 'reg_lambda': 0.01403433319157632}. Best is trial 1 with value: 19.051068389649682.
[I 2026-02-23 18:07:28,418] Trial 3 finished with value: 18.91346792100349 and parameters: {'n_estimators': 118, 'max_depth': 7, 'learning_rate': 0.26250033751646434, 'subsample': 0.8491385865062208, 'colsample_bytree': 0.9520037836418482, 'reg_alpha': 0.2804290389107794, 'reg_lambda': 0.7539788959110618}. Best is trial 3 with value: 18.91346792100349.


Best trial: 3. Best value: 18.9135:  17%|███████▊                                       | 5/30 [00:00<00:01, 16.07it/s]

...

Best trial: 10. Best value: 16.6269:  97%|███████████████████████████████████████████▌ | 29/30 [00:02<00:00, 10.43it/s]

[I 2026-02-23 18:07:30,958] Trial 26 finished with value: 18.939642353810022 and parameters: {'n_estimators': 230, 'max_depth': 10, 'learning_rate': 0.04737499322903971, 'subsample': 0.676994813751712, 'colsample_bytree': 0.9339187305210672, 'reg_alpha': 0.991586551461278, 'reg_lambda': 0.3618507078991948}. Best is trial 10 with value: 16.626852979148953.
[I 2026-02-23 18:07:31,051] Trial 27 finished with value: 18.941985854841093 and parameters: {'n_estimators': 281, 'max_depth': 7, 'learning_rate': 0.10486935307218603, 'subsample': 0.7117360437403885, 'colsample_bytree': 0.9738121363234794, 'reg_alpha': 0.3099412739555405, 'reg_lambda': 0.9852551138464918}. Best is trial 10 with value: 16.626852979148953.
[I 2026-02-23 18:07:31,116] Trial 28 finished with value: 18.949976069050276 and parameters: {'n_estimators': 179, 'max_depth': 8, 'learning_rate': 0.16266371426577184, 'subsample': 0.8252781733283722, 'colsample_bytree': 0.7715830762366933, 'reg_alpha': 0.593082927183343, 'reg_lamb

Best trial: 10. Best value: 16.6269: 100%|█████████████████████████████████████████████| 30/30 [00:03<00:00,  9.93it/s]


[I 2026-02-23 18:07:31,198] Trial 29 finished with value: 16.63274461294272 and parameters: {'n_estimators': 304, 'max_depth': 5, 'learning_rate': 0.010647519980040336, 'subsample': 0.6368278191635873, 'colsample_bytree': 0.8232548367048188, 'reg_alpha': 0.8535172377005464, 'reg_lambda': 0.32662066290270986}. Best is trial 10 with value: 16.626852979148953.
Best params: {'n_estimators': 270, 'max_depth': 10, 'learning_rate': 0.010467163140623154, 'subsample': 0.6094239801651857, 'colsample_bytree': 0.848123592341211, 'reg_alpha': 0.6570101044155773, 'reg_lambda': 0.6037229095306418}
Best RMSE on validation: 16.626852979148953


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.848123592341211, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.010467163140623154,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=270, n_jobs=None,
             num_parallel_tree=None, ...)